## Set Up Environment

In [ ]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [ ]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)
with open("keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("keystores/staking_test.json", key_data)

In [ ]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

In [ ]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

### Send ICX to Balanced test wallet

In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server
# for the first time since reinitializing.

# send_icx(wallets[20].get_address(), 2000 * ICX, btest_wallet)

# transaction = TransactionBuilder()\
#     .from_(wallet.get_address())\
#     .to(btest_wallet.get_address())\
#     .value(1000000 * ICX)\
#     .step_limit(1000000) \
#     .nid(NID) \
#     .nonce(2) \
#     .version(3) \
#     .build()
# signed_transaction = SignedTransaction(transaction, wallet)
# tx_hash = icon_service.send_transaction(signed_transaction)
# tx_hash

### Deployed to testnet

In [ ]:
# Cell 5
# The following addresses are those deployed to the testnet.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx89f109f7898d7e44f0caeab2b88f8fdd0b7b56ea'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx26e0721cc7925836e0b5653d87348fd3f5a765f4'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx77dcbc1852aa3495dd472ca9a4486566e63ed007'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx118756bcbf8d8b8f6aa6941148ea8a7863d9ca91'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxe520166c4a06b9929ce71377df165264acab341f'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxa740cf5c443feb23491c5d24d75086dd576c2fa3'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cxcbf7b3994cbda4dcdfa9be610e20782f8171fcb0'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxbcc157601376f60367851f8f17899c0a2c05fca5'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxec5403c48597577ebcc630e28e7d29610f24e7d5'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx62b52d35a46b6a8bd4f741eb826c1403bc2227cc'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx8b3b733fa6288d4942d303f6faa315958ac07e60'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cxb883c2b2496dde7512ee4594de59cb0c230ea5b1'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx9aca494966a68ddbcb507915366d75adb37ba6ca'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cxec33a32bb9f296edbb3f57a451b8ae36de1bbec2'}}

In [ ]:
# Cell 5a
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa0f715fb2c4bc8f4c6399c2cc26167a27be0aa61'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxbabed822d59b605dbeb6322735c529b292baac3b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx27aa3bf62145822e60d85fa5d18dabdcff5b9ada'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx01eee12b6614e5328e0a84261652cb7f055e0176'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd7b3e71dcff3d75392216e208f28ef68e8a54ec0'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx799f724e02560a762b5f2bd3b6d2d8d59d7aecc1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [ ]:
print(json.dumps({contract: contracts[contract]['SCORE'] for contract in contracts}))

## Deploy All SCOREs

In [ ]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        deploy.remove('bnXLM')
        deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "",  # ICX Station
            "",  # iBriz-ICONOsphere
            "",  # Mousebelt
            "",  # Parrot 9
            ""}  # ICONDAO
    else:
        return

    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate', 'params': {'_delegations': [{'_address': prep, '_votes_in_per': 100 * ICX // len(preps)} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    print(result)
    return result


In [ ]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

In [ ]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

In [ ]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

In [ ]:
# Cell 7c
# Set up and fund markets



In [ ]:
# Cell 7d
# Reset all addresses through governance.

# addresses = {
#     "loans": "cx89f109f7898d7e44f0caeab2b88f8fdd0b7b56ea",
#     "dex": "cxcbf7b3994cbda4dcdfa9be610e20782f8171fcb0",
#     "staking": "cx26e0721cc7925836e0b5653d87348fd3f5a765f4",
#     "rewards": "cxa740cf5c443feb23491c5d24d75086dd576c2fa3",
#     "reserve": "cx118756bcbf8d8b8f6aa6941148ea8a7863d9ca91",
#     "dividends": "cx77dcbc1852aa3495dd472ca9a4486566e63ed007",
#     "daofund": "cxe520166c4a06b9929ce71377df165264acab341f",
#     "oracle": "cx61a36e5d10412e03c907a507d1e8c6c3856d9964",
#     "sicx": "cxec5403c48597577ebcc630e28e7d29610f24e7d5",
#     "bnUSD": "cx62b52d35a46b6a8bd4f741eb826c1403bc2227cc",
#     "baln": "cx9aca494966a68ddbcb507915366d75adb37ba6ca",
#     "bwt": "cxec33a32bb9f296edbb3f57a451b8ae36de1bbec2"}

# send_tx('governance', 0, 'setAddresses', {'_addresses': addresses}, btest_wallet)
# send_tx('governance', 0, 'setContractAddresses', {}, btest_wallet)


## Deploy or Update a single SCORE

In [ ]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'rewards'
update = 1
params = {}
if update == 0:
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    deploy_SCORE(contract, params, btest_wallet, update)


In [ ]:
# Update all

compress()
for contract in contracts:
    deploy_SCORE(contract, {}, btest_wallet, 1)

## Test of Typical Flow

In [ ]:
# Cell 9
# 1. staking.stakeICX() - Get 800 ICX worth of sICX from Staking
# 2. loans.depositAndBorrow() - Deposit 800 ICX and Mint 50 bnUSD Loan
# 3. loans.depositAndBorrow() - Just deposit 30 ICX collateral
# 4. loans.depositAndBorrow() - Originate Loan of 10 bnUSD
# 5. loans.returnAsset() - Repay 5 bnUSD
# 6. loans.withdrawCollateral() - Withdraw 25 sICX Collateral
# 7. depositAndBorrow() via sICX.transfer() - Add 200 sICX collateral directly
# Print results1 if there are any failures.

txns = [{'contract': 'staking', 'value': 800*ICX, 'method': 'stakeICX', 'params': {}},
        {'contract': 'loans', 'value': 800*ICX, 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': 50 * ICX}},
        {'contract': 'loans', 'value': 30*ICX, 'method': 'depositAndBorrow', 'params': {'_asset': '', '_amount': 0}},
        {'contract': 'loans', 'value': 0, 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': 10 * ICX}},
        {'contract': 'loans', 'value': 0, 'method': 'returnAsset', 'params': {'_symbol': 'bnUSD', '_value': 5 * ICX}},
        {'contract': 'loans', 'value': 0, 'method': 'withdrawCollateral', 'params': {'_value': 25 * ICX}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['loans']['SCORE'], '_value': 200 * ICX, '_data': json.dumps({"_asset": "", "_amount": 0}).encode()}}]

results1 = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
    results1[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

### Add new assets

In [ ]:
# Not totally working yet since the Oracle needs to be set through the governance SCORE.
# send_tx('governance', 0, 'addAsset', {'_token_address': contracts['bnDOGE']['SCORE'], '_active': True, '_collateral': False}, btest_wallet)
# send_tx('governance', 0, 'addAsset', {'_token_address': contracts['bnXLM']['SCORE'], '_active': True, '_collateral': False}, btest_wallet)
# send_tx('bnDOGE', 0, 'setOracle', {'_address': contracts['oracle']['SCORE']}, btest_wallet)
# send_tx('bnXLM', 0, 'setOracle', {'_address': contracts['oracle']['SCORE']}, btest_wallet)


In [ ]:
call_tx('loans', 'getDebts', {'_address_list': [btest_wallet.get_address(), wallet.get_address()], '_day': 2})


### Test Liquidation

In [ ]:
# Cell 10
# 1. Deposit collateral to fresh wallet
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.
# 3a. Mints 200 bnUSD to the test address without checking collateralization ratio
#     With the above collateral deposit it will put the position in a standing of Liquidation.
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' bnUSD and have standing of 'Liquidate'.

send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
send_tx('loans', 170 * ICX, 'depositAndBorrow', {'_asset': '', '_amount': 0}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'create_test_position', {'_address': btest_wallet.get_address(), '_asset': 'bnUSD', '_amount': 200 * ICX}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)


In [ ]:
# Cell 11
# 4a. If the account does not yet have a standing of 'Liquidate' add another loan of '0xad78ebc5ac6200000'.

send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
send_tx('loans', 1, 'create_test_position', {'_address': btest_wallet.get_address(), '_asset': 'bnUSD', '_amount': 200 * ICX}, btest_wallet)
send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
call_tx('loans', 'checkDeadMarkets', {})
print(call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()}))


In [ ]:
# Cell 12
# 5. Check the account standing after depositing collateral - Should be 'No Debt'.
# 6. Liquidate the account position.
# 7. Check the standing of the account after liquidation. Should now have zero balance for sICX and bnUSD.
# 8. Checking the debts should show the sum of the borrower debt and the bad debt equals the total supply.

call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'liquidate', {'_owner': btest_wallet.get_address()}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})


## Retire some debt to get bnUSD out of Dead Market state

In [ ]:
# Cell 13
# 1. Check that there is a Dead Market
# 2. Check the bad debt, liquidation pool and dead_market state of the asset.
# 3. Retire enough of the bad debt to get the asset out of the dead_market state.
# 4. Again, check the bad debt, liquidation pool and dead_market state of the asset.

call_tx('loans', 'checkDeadMarkets', {})
call_tx('loans', 'getAvailableAssets', {})
send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 200 * ICX}, btest_wallet)
call_tx('loans', 'getAvailableAssets', {})


### Unstake some sICX

In [ ]:
send_tx('sicx', 0, 'transfer', {'_to': contracts['staking']['SCORE'], '_value': 50 * ICX, '_data': json.dumps({"method": "unstake", "params": {}}).encode()}, btest_wallet)

## Tests with multiple wallets and redemptions

In [ ]:
# Create a set of wallets

# wallets = []
# for i in range(200):
#     wallets.append(KeyWallet.create())
# with open("../wallets.pkl", 'wb') as f:
#     pkl.dump(wallets, f)

In [ ]:
with open("../wallets.pkl", 'rb') as f:
    wallets = pkl.load(f)

In [ ]:
for i in range(20):
    print(f'{wallets[i].get_address()}')

In [ ]:
for i in range(100):
    send_icx(wallets[i].get_address(), 2000 * ICX, btest_wallet)

In [ ]:
for i in range(100):
    print(f'{i:3d}: {wallets[i].get_address()}: {icon_service.get_balance(wallets[i].get_address()) / 10**18: 9.3f}')

In [ ]:
for i in range(20):
    print(int(call_tx("bnUSD", "balanceOf", {"_owner": wallets[i].get_address()}), 0) / 10**18)

In [ ]:
# 10000 ICX into each wallet. Uncomment if needed. These wallets are already more than well stocked enough on the tbears server.

# for i in range(20):
#     transaction = TransactionBuilder()\
#         .from_(wallet.get_address())\
#         .to(wallets[i].get_address())\
#         .value(10 * ICX)\
#         .step_limit(1000000) \
#         .nid(NID) \
#         .nonce(2) \
#         .version(3) \
#         .build()
#     signed_transaction = SignedTransaction(transaction, wallet)
#     tx_hash = icon_service.send_transaction(signed_transaction)
# tx_hash

In [ ]:
# Mint some bnUSD to first 70 wallets.

for i in range(70):
    res = send_tx('loans', 500 * ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 100 * ICX}, wallets[i])


In [ ]:
4458.523176224666700000 / 0.461900911630734200

In [ ]:
# Drive rewards distributions

for i in range(15):
    res = send_tx('loans', 0, 'depositAndBorrow', {'_asset': '', '_amount': 0}, wallets[i])


In [ ]:
res = send_tx('loans', 500 * ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 100 * ICX}, wallets[90])

In [ ]:
for i in range(15, 20):
    res = send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 15 * ICX}, wallets[i+100])
    sleep(1)

In [ ]:
# redeem 10 bnUSD from each of 20 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.

for i in range(10):
    params = {'_to': wallets[i+100].get_address(), '_value': 20 * ICX}
    res = send_tx('bnUSD', 0, 'transfer', params, wallets[i])
    sleep(2)
    res = send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 15 * ICX}, wallets[i+100])
    sleep(1)

In [ ]:
for _ in range(20):
    send_tx('loans', 0, 'depositAndBorrow', {'_asset': '', '_amount': 0}, btest_wallet)
    sleep(1)

In [ ]:
# Gets the total number of borrowers in the system and iterates over them by index to update each one.

total_borrowers = int(call_tx('loans', 'borrowerCount', {}), 0)
for i in range(total_borrowers):
    params = {'_index': i + 1}
    params = {'_owner': call_tx('loans', 'getPositionAddress', params)}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Mint some bnUSD to each wallet

for i in range(80, 110):
    params = {'_asset': 'bnUSD', '_amount': 100 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(1200 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# redeem 50 bnUSD from each of 20 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.

for i in range(80, 110):
    params = {'_to': wallets[i+50].get_address(), '_value': 50 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['bnUSD']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)
    params = {"method": "_retire_asset", "params": {}}
    data = json.dumps(params).encode("utf-8")
    params = {'_to': contracts['loans']['SCORE'], '_value': 50 * ICX, '_data': data}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i+50].get_address())\
        .to(contracts['bnUSD']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i+50])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for _ in range(10):
    params = {'_index': 1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionAddress")\
                        .params(params)\
                        .build()
    params = {'_owner': icon_service.call(call)}
    transaction = CallTransactionBuilder()\
        .from_(wallets[1].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[1])
    tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(100):
    params = {'_owner': wallets[i].get_address()}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_holders': [wallet.get_address(), btest_wallet.get_address()]}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
i = 0
{i: [key, result[key]] for i, key in enumerate(result.keys())}

In [ ]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("borrowerCount")\
                    .params(params)\
                    .build()
borrowerCount = int(icon_service.call(call), 0)

addresses = []
for i in range(1, borrowerCount + 1):
    params = {'_index': i, '_day': -1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionByIndex")\
                        .params(params)\
                        .build()
    position = icon_service.call(call)
    addresses.append(position['address'])

params = {'_holders': addresses}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
holders = icon_service.call(call)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt', 'dex', 'daofund']:
    params = {'_owner': contracts[contract]['SCORE']}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['baln']['SCORE'])\
                        .method("balanceOf")\
                        .params(params)\
                        .build()
    result = int(icon_service.call(call), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

i = 0
holdings = {i: [key, int(holders[key], 0), int(holders[key], 0) / 10**18] for i, key in enumerate(holders.keys())}
total = 0
for key in holdings:
    total += holdings[key][1]
    print(f'{holdings[key]}')

print(f'Total unclaimed: {total / 10**18}')
print(baln_balances)
print(f'Total BALN: {total_balances / 10**18}')

In [ ]:
params = {}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['rewards']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("claimRewards")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

print(i)
res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(0, 70):
    params = {}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['rewards']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("claimRewards")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_name': 'BALN/bnUSD', '_snapshot_id': 1, '_limit': 10, '_offset': 0}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("getDataBatch")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
icon_service.get_balance(wallet.get_address()) / 10**18

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['baln']['SCORE'])\
                        .method('availableBalanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['sicx']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wall in [wallet, btest_wallet]:
    address = wall.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['baln']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = int(result, 0) // 10**18
print(balances)

In [ ]:
send_tx('baln', 0, 'transfer', {'_to': 'hx7e02f538732a93893cb225de3e1147a1611ef0ea', '_value': 10**23}, btest_wallet)

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [ ]:
contracts['dex']['SCORE']

In [ ]:
# Cell 26

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1000 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 27

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 28

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolStats')\
                    .params({'_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 29

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("stakeICX")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 30

data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(30000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 31

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 32

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'sICX/ICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

### sICX / bnUSD Pool

In [ ]:
# Create a new pool and generate assets for it

sICX_amount = 16 * ICX
bnUSD_amount = 50 * ICX
icx_price = 2
sICX_contribute_amount = int(10 * ICX)
bnUSD_contribute_amount = int(25 * ICX)

txns = [{'contract': 'staking', 'value': sICX_amount, 'method': 'stakeICX', 'params': {}},
        {'contract': 'loans', 'value': int(icx_price * 6 * bnUSD_amount), 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': bnUSD_amount}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

In [ ]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolStats')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Add assets to existing pool

sICX_contribute_amount = int(2 * ICX)
bnUSD_contribute_amount = int(2 * ICX)

txns = [{'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

In [ ]:
# Cell 35

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 36

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['baln']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 37

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['baln']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
wallets[6].get_address()

In [ ]:
# Cell 38a

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['baln']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 10 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(5, 10):
    data = "{\"method\": \"_deposit\"}".encode("utf-8")
    params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
    send_tx('baln', 0, 'transfer', params, wallets[i])
    data = "{\"method\": \"_deposit\"}".encode("utf-8")
    params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
    send_tx('bnUSD', 0, 'transfer', params, wallets[i])
    params = {'_baseToken': contracts['baln']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 10 * ICX}
    send_tx('dex', 0, 'add', params, wallets[i])

In [ ]:
# Cell 38b

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_baseValue': 2 * ICX, '_quoteValue': 1 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 39

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("dexPermit")\
    .params({'_id': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 40

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getInversePrice')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**10

In [ ]:
# Cell 41

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 42

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['bnUSD']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['bnUSD']['SCORE']))
print(result)

In [ ]:
# Cell 43

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':2, '_name': 'sICX/bnUSD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':3, '_name': 'BALN/bnUSD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Set the percentages to use with the BALN/bnUSD pool included.

RECIPIENTS = [{'recipient_name': 'Loans', 'dist_percent': 25 * 10**16},
              {'recipient_name': 'sICX/ICX', 'dist_percent': 10 * 10**16},
              {'recipient_name': 'sICX/bnUSD', 'dist_percent': 175 * 10**15},
              {'recipient_name': 'BALN/bnUSD', 'dist_percent': 175 * 10**15},
              {'recipient_name': 'Worker Tokens', 'dist_percent': 20 * 10**16},
              {'recipient_name': 'Reserve Fund', 'dist_percent': 5 * 10**16},
              {'recipient_name': 'DAOfund', 'dist_percent': 5 * 10**16}]

send_tx('governance', 0, 'updateBalTokenDistPercentage', {'_recipient_list': RECIPIENTS}, btest_wallet)

In [ ]:
# Cell 45

to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":190}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': ICX // 100, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 45

to_token = contracts['bnUSD']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': ICX // 100, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 46

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 47

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 48

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('totalSupply')\
                    .params({'_id': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 49

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_id':2, '_value': 1 * ICX, '_withdraw': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_id': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 50

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 30 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 51

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 52

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("cancelSicxicxOrder")\
    .params({}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
to_token = contracts['baln']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')